## Realigning a Toastmasters District Using a Genetic Algorithm

This notebook takes cleaned district data and uses the DEAP library to realign the district into areas.

The fitness function optimizes the variation (standard deviation) of the club quality scores, minimizes the normalized average distance between the clubs, and optimizes the number of clubs which must be 4-6 with an ideal of 5.

The club quality score is equal parts:
1. A normalized absolute variation from 25 members - minimized,
2. A normalized awards per member (participation) - maximized, and
3. A normalized absolute variation of new members from 20% (retention) - minimized.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import contextily as ctx
from itertools import combinations
import random

# Prepare Data

In [2]:
# Import data - only quality score needed for algorithm
clubs = pd.read_csv('clubs_to_realign.csv')
clubs.head()

,club_no,n_quality
0,5509,0.677709
1,7036,0.695792
2,9682,0.614715
3,584009,0.475900
4,1100434,0.768516


We'll be using the distance matrix we created earlier. Let's import it and set it up.

In [3]:
dist = pd.read_csv('club_distance_matrix.csv')
dist.set_index('club_no', inplace=True)
# Make sure the columns are integers too
dist.columns = dist.columns.astype(int)
dist.head()

,5509,7036,9682,584009,1100434,718,4819,9790,5069647,7575630,...,1783,596735,4700632,5258000,2690,8569,3929213,4822437,5569,1565753
club_no,,,,,,,,,,,,,,,,,,,,,
5509,0.000000,0.000000,0.309451,0.315310,0.000000,0.309451,0.338685,0.297754,0.347098,0.311623,...,0.752496,0.737553,0.748308,0.748308,0.748079,0.748308,0.748308,0.749572,0.748308,0.748308
7036,0.000000,0.000000,0.309451,0.315310,0.000000,0.309451,0.338685,0.297754,0.347098,0.311623,...,0.752496,0.737553,0.748308,0.748308,0.748079,0.748308,0.748308,0.749572,0.748308,0.748308
9682,0.360002,0.360002,0.000000,0.033326,0.360002,0.000000,0.035332,0.024728,0.100703,0.011137,...,0.896155,0.892741,0.894353,0.894353,0.893528,0.894353,0.894353,0.895944,0.894353,0.894353
584009,0.378986,0.378986,0.034432,0.000000,0.378986,0.034432,0.047424,0.024879,0.070442,0.045275,...,0.927359,0.924798,0.926084,0.926084,0.925535,0.926084,0.926084,0.927159,0.926084,0.926084
1100434,0.000000,0.000000,0.309451,0.315310,0.000000,0.309451,0.338685,0.297754,0.347098,0.311623,...,0.752496,0.737553,0.748308,0.748308,0.748079,0.748308,0.748308,0.749572,0.748308,0.748308


Since we're optimizing standard deviation, let's consider what an optimum would be. Ideally, there would be a good variety of strong and weak clubs, not all strong or all weak. All of one or the other would be a standard deviation of 0. Half strong and half weak would be 0.5 (from a range of 0 to 1). The ideal is something in between.

In [4]:
# This is a hypothetical area as a list of club scores
# ranging from our normalized 0 to 1 scale. 
# This would be a well-balanced area.
ideal = [0.0, 0.25, 0.5, 0.75, 1.0]
# Calculate ideal standard deviation.
std = round(np.std(ideal), 3)
print(f'An ideal distribution: {std}')

An ideal distribution: 0.354


### Functions to Group and Format Areas

DEAP can only register function names so all additional formatting needs to be done before that.

Since we want all areas to be 5 clubs, if we end up with less than 4 we need to make the last 2-4 areas have 4 clubs instead. Here we make a district encoded as a list of areas with lists of clubs. Input is a dataframe of clubs. Set clubs to default so that we don't need to add the parameter when registering it in the DEAP toolbox. We will also use the grouping function during crossover so we need a randomizing parameter to be able to turn it off at that time. 

In [5]:
def group_areas(clubs=clubs, randomize=False):
    if randomize == True:
        random.shuffle(clubs)
    
    # Check how many clubs would be left in last area
    if len(clubs) % 5 in [0, 4]:
        for i in range(0, len(clubs), 5): 
            yield clubs[i:i + 5]
        
    elif len(clubs) % 5 == 3:
        for i in range(0, len(clubs)-8, 5):
            yield clubs[i:i + 5]
        for i in range(len(clubs)-8, len(clubs), 4):
            yield clubs[i:i + 4]
        
    elif len(clubs) % 5 == 2:
        for i in range(0, len(clubs)-12, 5):
            yield clubs[i:i + 5]
        for i in range(len(clubs)-12, len(clubs), 4):
            yield clubs[i:i + 4]
                
    else: # Remaining cases have 1 left over club
        for i in range(0, len(clubs)-16, 5):
            yield clubs[i:i + 5]
        for i in range(len(clubs)-16, len(clubs), 4):
            yield clubs[i:i + 4]

def areas_list(clubs=clubs):
    return list(group_areas(list(clubs['club_no']), randomize=True))

Test our areas_list function on a list of our clubs. Notice the last two areas have 4 clubs each.

In [6]:
district1 = areas_list(clubs)
district1

[[5569, 5042512, 4107, 7059, 5869106],
 [2427, 953233, 1783, 1047602, 4858],
 [6071, 1036600, 9354, 4015, 6632930],
 [1412885, 6861322, 1176566, 1190, 9214],
 [1331602, 5545568, 730163, 9682, 7274],
 [1535564, 2556863, 7031829, 1171779, 8941],
 [2189079, 1526701, 8569, 9161, 3318],
 [9469, 9019, 1595518, 7463287, 6613239],
 [6661, 2690, 1165752, 7533, 3372438],
 [614471, 997315, 3549, 9790, 7479372],
 [6715494, 2808, 713, 6142, 7817],
 [7022029, 5918, 6654663, 8552, 7532701],
 [4819, 942489, 4533, 845547, 8041],
 [1291183, 9598, 1171849, 4154, 3356972],
 [5981, 5112712, 607240, 1995527, 718],
 [7034388, 2923054, 3063370, 4718634, 596735],
 [967, 2146, 6970706, 7554675, 8169],
 [695532, 7384295, 7402713, 8952, 5055],
 [7327347, 4793192, 4110, 7036, 3431353],
 [6820584, 6891000, 992874, 437, 5509],
 [7479409, 1565753, 8412, 5859633, 8853],
 [3859, 1582927, 7881, 3401898, 5258000],
 [1158551, 2364, 5736, 5928, 3408653],
 [4918271, 8363, 3074518, 584516, 6590],
 [8983, 584009, 3240871, 409

### Build evaluation function
- We will make sure most areas have 5 clubs through the `areas_list` function. 
- Our next criterion is minimizing the distance from the ideal quality distribution.
- The final criterion is minimizing the distance between clubs.
- Write functions for both calculations and call them both in the evaluation function.
- Keep the two functions separate in case we decide to change the weights later.
- Where parameters are constant (i.e. clubs or dist), set them as the default to work with the DEAP library better. 

In [7]:
# Function to calculate quality distribution
# Specifically, we're going to minimize the average distance 
# of the standard deviation from 0.35 which is an approximately 
# ideal quality distribution of 5 clubs

# This function returns a list of quality scores from a list of club numbers
def area_qualities(area, clubs=clubs):
    return [clubs[clubs['club_no'] == club]['n_quality'].iloc[0] for club in area]

# This function returns the average distance from the ideal distribution for the district
def quality_std(district, clubs=clubs):
    return sum([abs(np.std(area_qualities(area, clubs))-0.35) for area in district]) / len(district)

In [8]:
# Function to calculate average distance between all clubs in each area for the district

# This function calculuates average distance in one area.
def area_dist(clubs, dist=dist):
    return sum([dist.loc[pair] for pair in list(combinations(clubs, 2))]) / \
            len(list(combinations(clubs, 2)))

# This function calculates average distance of all areas
def district_dist(district, dist=dist):
    return sum([area_dist(area, dist) for area in district]) / len(district)

In [33]:
# Evaluation Function
# This returns a score representing the quality distribution
# and average physical distance between clubs
def evaluate_district(district, clubs=clubs, dist=dist):
    return (quality_std(district, clubs) + district_dist(district, dist)) / 2

In [34]:
evaluate_district(district1)

0.2189403187434103

### Build selection function
- We will use tournament selection which means we'll use the evaluation function and pick the best of some number of districts (default k=3) to pick as a parent for the next generation.
- Using 4 tends to converge on the initial best performers too soon and 2 tends to prevent convergence.
- Additionally, we'll want to keep track of the best from generation to generation (Hall of Fame).
- In the main algorithm, we'll repeat this until we have the number of parents we need.

In [49]:
def t_select(pop, k=3):
    competitors = random.sample(pop, k)
    best_district = []
    best_score = 1
    for i in competitors:
        score = evaluate_district(i)
        # Recall we want the lowest score
        if score < best_score:
            best_score = score
            best_district = i
    return best_district

In [50]:
t_select(population[:25])

[[6990556, 9469, 5055, 1036600, 4858],
 [4157985, 695532, 7034388, 2556863, 5981],
 [953233, 3929213, 7274, 997315, 730163],
 [4110, 6786, 3395235, 5553533, 7036],
 [9598, 5069647, 3431353, 1331602, 2189079],
 [3240871, 4721, 1526701, 8363, 4446],
 [6820584, 8952, 6142, 3356972, 9682],
 [718, 596735, 3401898, 4918271, 6861322],
 [437, 8041, 5869106, 2876291, 1207],
 [1408278, 7031829, 1047602, 1244830, 8941],
 [6590, 6970706, 4750107, 7587, 4533],
 [2364, 584516, 1412885, 1588444, 8552],
 [1463775, 2912, 5042512, 4154, 7479409],
 [9872, 5674, 6654663, 3372438, 2146],
 [6632930, 7022029, 1158551, 1100434, 7479372],
 [4108, 7402713, 6644914, 5509, 5736],
 [4786679, 2808, 1535564, 6891000, 9354],
 [8853, 1995527, 5112712, 4095, 8569],
 [8169, 4801055, 9161, 9790, 3063370],
 [2038660, 4015, 713, 3859, 583467],
 [3549, 6975086, 1595518, 8631, 7306126],
 [942489, 7327347, 5928, 6380, 9214],
 [6887806, 1176575, 1171779, 7817, 3074518],
 [2427, 7554675, 7059, 5918, 1565753],
 [4182, 4819, 4700

### Build Mating Function
- Here we'll implement a variation of the one-point crossover. 
- Offspring 1 carries forward the first cut of parent 1.
- Add any remaining complete areas from parent 2. 
- Fill in the remaining areas with missing clubs sequenced from parent 1.
- Repeat for offspring 2.

Create a helper flattening function.

In [10]:
def flatten(district):
    return [club for area in district for club in area]

# Regroup: list(group_areas(clubs_list))

In [11]:
# Create another parent for testing
district2 = areas_list(clubs)
district2

[[5859633, 6632930, 607240, 2912, 1582927],
 [614471, 2808, 4822437, 4182, 4786679],
 [584009, 1176566, 6644914, 7587, 4918271],
 [7059, 9872, 5258000, 3549, 6071],
 [6970706, 695532, 1463775, 953233, 7034388],
 [7274, 1176575, 3395235, 5928, 1244830],
 [5553533, 8952, 997315, 8853, 8941],
 [5674, 3372438, 6661, 2427, 713],
 [3859, 8983, 992874, 6820584, 1588444],
 [1171849, 3812934, 7532701, 9598, 1036600],
 [3074518, 8552, 4700632, 3063370, 4819],
 [4015, 5569, 9682, 8041, 7463287],
 [4718634, 7402713, 4446, 1565753, 596735],
 [7036, 1783, 6891000, 584516, 6654663],
 [4107, 2690, 2876291, 4721, 3356972],
 [6590, 6887806, 1331602, 7479409, 3408653],
 [718, 942489, 5981, 3318, 6786],
 [3240871, 7031829, 845547, 4793192, 630505],
 [1526701, 4108, 8569, 4533, 1995527],
 [9469, 6380, 8412, 6613239, 7327347],
 [3761504, 1100434, 5545568, 2038660, 5869106],
 [7479372, 5736, 6754191, 7306126, 2189079],
 [9019, 4801055, 1165752, 2364, 7881],
 [1158551, 7022029, 5055, 4750107, 1291183],
 [8169

In [12]:
# Mating function
def group_crossover(p1, p2):
    # Find the cut-point
    cut = int(len(p1)/2)
    # Make ordered lists of club numbers
    p1_list = flatten(p1)
    p2_list = flatten(p2)
    
    # Initialize offspring
    o1_cut = p1[:cut]
    o2_cut = p2[:cut]
    
    # Add any remaining valid areas from second parent
    p2_rem = [area for area in p2 if not any(club in area for club in flatten(o1_cut))]
    p1_rem = [area for area in p1 if not any(club in area for club in flatten(o2_cut))]

    o1_valid = o1_cut + p2_rem
    o2_valid = o2_cut + p1_rem

    # Add missing clubs
    o1_missing = [club for club in p1_list if club not in flatten(o1_valid)]
    o2_missing = [club for club in p2_list if club not in flatten(o2_valid)]
    
    o1 = o1_valid + list(group_areas(o1_missing))
    o2 = o2_valid + list(group_areas(o2_missing))
    
    return o1, o2


### Mutation Function
Like the crossover mating function, we need a custom mutation function since I didn't see a swap mutation function.

In [17]:
def swap_mutate(district):
    d_list = flatten(district)

    club_index_1 = random.randint(0, len(d_list))
    club_index_2 = random.randint(0, len(d_list))
    
    club_1 = d_list[club_index_1]
    club_2 = d_list[club_index_2]
    
    d_list[club_index_1] = club_2
    d_list[club_index_2] = club_1
    
    return list(group_areas(d_list))


### Population Sorting, Stats, and Hall of Fame
- During the main algorithm, we'll need to sort the population to retain the best.
- We will want to visualize convergence which can be seen plotting the average and minimum scores as they approach zero across the generations.
- The Hall of Fame keeps the best individuals without risking crossover or mutation.

In [85]:
def sort_districts(districts):
    # Append the scores at the beginning of each district
    scored_districts = [[evaluate_district(d)] + d for d in districts]
    # Sort ascending according to the scores (lowest first)
    sorted_districts = sorted(scored_districts)
    # Return the districts without the scores
    return [d[1:] for d in sorted_districts]

In [86]:
def average_district_score(districts):
    return np.mean([evaluate_district(d) for d in districts])

def best_district_score(districts):
    return np.min([evaluate_district(d) for d in districts])

def hall_of_fame(districts, size=10):
    return sort_districts(districts)[:size]

### Test Functions

In [117]:
# Make some toy districts and print out their scores for comparison
d1 = areas_list(clubs[:24])
d2 = areas_list(clubs[:24])
d3 = areas_list(clubs[:24])
d4 = areas_list(clubs[:24])
d5 = areas_list(clubs[:24])
toy_pop = [d1, d2, d3, d4, d5]
[evaluate_district(d) for d in toy_pop]

[0.36224413954893453,
 0.36158858928970394,
 0.3454328740857442,
 0.3203331627715555,
 0.33008569134214444]

In [90]:
# Test each stats function
print(f'Average district score: {average_district_score(toy_pop)}')
print(f'Best district score: {best_district_score(toy_pop)}')
print(f'Top 3 best districts: {[evaluate_district(d) for d in hall_of_fame(toy_pop, size=3)]}')

Average district score: 0.3649441261969918
Best district score: 0.341869716828055
Top 3 best districts: [0.341869716828055, 0.3494228829947085, 0.37329572875325434]


In [101]:
# Test Crossover
o1, o2 = group_crossover(district1, district2)
o1, o2

([[5569, 5042512, 4107, 7059, 5869106],
  [2427, 953233, 1783, 1047602, 4858],
  [6071, 1036600, 9354, 4015, 6632930],
  [1412885, 6861322, 1176566, 1190, 9214],
  [1331602, 5545568, 730163, 9682, 7274],
  [1535564, 2556863, 7031829, 1171779, 8941],
  [2189079, 1526701, 8569, 9161, 3318],
  [9469, 9019, 1595518, 7463287, 6613239],
  [6661, 2690, 1165752, 7533, 3372438],
  [614471, 997315, 3549, 9790, 7479372],
  [6715494, 2808, 713, 6142, 7817],
  [7022029, 5918, 6654663, 8552, 7532701],
  [4819, 942489, 4533, 845547, 8041],
  [1291183, 9598, 1171849, 4154, 3356972],
  [5981, 5112712, 607240, 1995527, 718],
  [7034388, 2923054, 3063370, 4718634, 596735],
  [967, 2146, 6970706, 7554675, 8169],
  [3859, 8983, 992874, 6820584, 1588444],
  [695532, 7384295, 7402713, 8952, 5055],
  [7327347, 4793192, 4110, 7036, 3431353],
  [6891000, 437, 5509, 7479409, 1565753],
  [8412, 5859633, 8853, 1582927, 7881],
  [3401898, 5258000, 1158551, 2364, 5736],
  [5928, 3408653, 4918271, 8363, 3074518],
  [

In [19]:
# Test Mutate
swap_mutate([[5569, 5042512, 4107, 7059, 5869106],
             [2427, 953233, 1783, 1047602, 4858]])

[[5569, 2427, 4107, 7059, 5869106], [5042512, 953233, 1783, 1047602, 4858]]

### Algorithm Constants

In [137]:
POPULATION_SIZE = 100
P_CROSSOVER = 0.5  # probability for crossover
P_MUTATION = 0.1   # probability for mutating an individual
MAX_GENERATIONS = 5
HOF_SIZE = 10 # Hall of Fame size

random.seed(42)

### Run Algorithm

In [140]:
def ga(population=POPULATION_SIZE, pc=P_CROSSOVER, pm=P_MUTATION, gen=MAX_GENERATIONS, hof=HOF_SIZE):
    
    # Initialize population
    pop = [areas_list() for _ in range(population)]
    
    # Initialize stats
    averages = [average_district_score(pop)]
    best_scores = [best_district_score(pop)]
    hof_list = hall_of_fame(pop, hof)
    
    # Generation Loop
    for g in range(gen):
        offspring = []
        
        # Selection and crossover
        for _ in range(len(pop)):
            p1 = t_select(pop)
            p2 = t_select(pop)
            
            if random.random() < pc:
                offspring += group_crossover(p1, p2)
            else:
                offspring += (p1 + p2)
            
        # Mutation
        offspring = [swap_mutate(d) if random.random() < pm else d for d in offspring]
        
        # Sort and stats
        offspring = sort_districts(offspring)
        averages += [average_district_score(offspring)]
        best_scores += [best_district_score(offspring)]
        hof_list = hall_of_fame(offspring, hof)
        
        print(f'Generation {g}: Average: {averages[-1]}, Best: {best_scores[-1]}')
        
        pop = offspring[:]
        
    return pop[0], averages, best_scores, hof_list


In [141]:
best, average, best_score, hof_list = ga()

TypeError: 'int' object is not iterable

### Visualize Stats

In [ ]:
sns.set_style("whitegrid")
plt.plot(minFitnessValues, color='red')
plt.plot(meanFitnessValues, color='green')
plt.xlabel('Generation')
plt.ylabel('Min / Average Fitness')
plt.title('Min and Average Fitness over Generations')
plt.show()

### Check Best Results

In [ ]:
print("Hall of Fame Individuals = ", *hof.items, sep="\n")
print("Best Ever Individual = ", hof.items[0])

### Export Solution(s)

In [ ]:
first_district = hof.items[0]
second_district = hof.items[1]
third_district = hof.items[2]